# Coursera Capstone Project

This is fulfilment of coursera course and will be mainly used for capstone project

In [64]:
#scientific library
import pandas as pd
import numpy as np

In [399]:
print("Hello Capstone Project Course!!")

Hello Capstone Project Course!


In [66]:
#web scraping library
import requests
from bs4 import BeautifulSoup
import time

In [155]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [67]:
#create a get request to a URL
r = requests.get('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto')

In [71]:
#extract content
c = r.content

In [73]:
# Create a soup object
soup = BeautifulSoup(c, 'lxml')

In [160]:
# Find the element on the webpage
main_content = soup.find('table', attrs = {'class': 'wikitable'})

In [163]:
#main_content

In [164]:
# Extract the relevant information 
content = main_content.find_all('a')

In [179]:
#empty list
cc = []

#loop through the content and get value of title attribute
for i in content: 
    cc.append(i['title'])


**NOTES** :
I used https://geocode.localfocus.nl/ to get coordinate values of cc (neighborhood scraped from wikipedia). The list values is copied and pasted to the website. Locations from cc shows multiple values of coordinates, for this project, the first occurence of suggested coordinates is chosen. The result of my query is downloaded and available at https://tinyurl.com/yyd4t85l

In [302]:
import os

#check current working directory
wd = os.getcwd()

#read csv file
df = pd.read_csv(wd + '/Documents/GitHub/ipynb_coursera/toronto.csv', header=None, decimal=",")

In [303]:
#add columns
df.columns = ['Neighborhood','Latitude','Longitude']


In [304]:
neighborhoods = df

In [305]:
neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,"Carleton Village Shore, NS, Canada",43.661584,-65.328253
1,"Corso Italia Garage, Toronto, ON, Canada",43.676946,-79.444991
2,"York, Toronto, ON, Canada",43.680189,-79.492232
3,"Toronto, ON, Canada",43.702303,-79.357423
4,"Brookhaven-Amesbury, Toronto, ON, Canada",43.701141,-79.485589


In [306]:
#neighborhoods['Latitude'] = pd.to_numeric(df['Latitude'])


In [307]:
neighborhoods.dtypes

Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

#### Use geopy library to get the latitude and longitude values of New York City.

In [308]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.653963, -79.387207.


Create a map of Toronto with neighborhoods superimposed on top.

In [310]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


#map_toronto

# add markers to map
for lat,lng,neighborhood in zip(neighborhoods['Latitude'],neighborhoods['Longitude'],neighborhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [311]:
CLIENT_ID = 'VS31FBLWDPVVEFZJILDIOQ4L4YOV5PUKNMWAAP23DSKTNRG0' # your Foursquare ID
CLIENT_SECRET = '0PF4BMKGQWD01UTSQ5XAKQX1EW3VDXFQUT2D4GQQB1QVAXHG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VS31FBLWDPVVEFZJILDIOQ4L4YOV5PUKNMWAAP23DSKTNRG0
CLIENT_SECRET:0PF4BMKGQWD01UTSQ5XAKQX1EW3VDXFQUT2D4GQQB1QVAXHG


In [312]:
df.loc[0, 'Neighborhood']

'Carleton Village Shore, NS, Canada'

Get the neighborhood's latitude and longitude values.

In [313]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Carleton Village Shore, NS, Canada are 43.661584000000005, -65.328253.


Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.

#### NOTE:  I used venue recommendation because fourssquare can't find area nearby . I added , openNow=1 & near=Toronta, CANadain

In [334]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&openNow=1&near=Toronto, Canada'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=VS31FBLWDPVVEFZJILDIOQ4L4YOV5PUKNMWAAP23DSKTNRG0&client_secret=0PF4BMKGQWD01UTSQ5XAKQX1EW3VDXFQUT2D4GQQB1QVAXHG&v=20180605&ll=43.661584000000005,-65.328253&radius=500&limit=100&openNow=1&near=Toronto, Canada'

Send the GET request and examine the resutls

In [335]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c8f8265f594df7cdf33231c'},
 'response': {'geocode': {'what': '',
   'where': 'toronto canada',
   'center': {'lat': 43.70011, 'lng': -79.4163},
   'displayString': 'Toronto, ON',
   'cc': 'CA',
   'geometry': {'bounds': {'ne': {'lat': 43.855547992182615,
      'lng': -79.1040939249301},
     'sw': {'lat': 43.58464308210616, 'lng': -79.64745469478004}}},
   'slug': 'toronto',
   'longId': '72057594044095801'},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 31,
  'suggestedBounds': {'ne': {'lat': 43.77346396146738,
    'lng': -79.33350094195907},
   'sw': {'lat': 43.63039880918509, 'lng': -79.429316580082}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227b

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [336]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [337]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,The Planet Traveler,Hostel,43.657202,-79.403568
2,Bar Buca,Italian Restaurant,43.643944,-79.399922
3,Fahrenheit Coffee,Coffee Shop,43.652384,-79.372719
4,Delta Hotels by Marriott Toronto,Hotel,43.642882,-79.383949


### Neighborhoods Exploration in Toronto

In [338]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [341]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )



Carleton Village Shore, NS, Canada
Corso Italia Garage, Toronto, ON, Canada
York, Toronto, ON, Canada
Toronto, ON, Canada
Brookhaven-Amesbury, Toronto, ON, Canada
Park, MB, Canada
Weston, Toronto, ON, Canada
Fairbank, Toronto, ON, Canada
Maple Leaf, Toronto, ON, Canada
Mount Dennis, Toronto, ON, Canada
Downsview, Toronto, ON, Canada
Humber Summit, Toronto, ON, Canada
Humbermede, Toronto, ON, Canada
Jane and Finch, Toronto, ON, Canada
York University Heights, Toronto, ON, Canada
New Toronto, Toronto, ON, Canada
Long Branch, Toronto, ON, Canada
North Mimico, Toronto, ON, Canada
Alderwood, Toronto, ON, Canada
The Queensway, Toronto, ON, Canada
The Kingsway, Toronto, ON, Canada
Etobicoke, Toronto, ON, Canada
Eringate-Centennial-West Deane, Toronto, ON, Canada
Toronto, ON, Canada
Humber Valley Village Junior Middle School, Toronto, ON, Canada
Islington-City Centre West, Toronto, ON, Canada
Markland Wood, Toronto, ON, Canada
Toronto, ON, Canada
Richview, Toronto, ON, Canada
Humber Heights, T

Let's check the size of the resulting dataframe

In [342]:
print(toronto_venues.shape)
toronto_venues.head()

(3092, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Corso Italia Garage, Toronto, ON, Canada",43.676946,-79.444991,Marcello's Pizzeria,43.678017,-79.442725,Italian Restaurant
1,"Corso Italia Garage, Toronto, ON, Canada",43.676946,-79.444991,Rio 40 Graus,43.677329,-79.446298,Brazilian Restaurant
2,"Corso Italia Garage, Toronto, ON, Canada",43.676946,-79.444991,King's Tacos,43.677681,-79.444445,Mexican Restaurant
3,"Corso Italia Garage, Toronto, ON, Canada",43.676946,-79.444991,Tre Mari Bakery,43.676666,-79.448740,Café
4,"Corso Italia Garage, Toronto, ON, Canada",43.676946,-79.444991,Pizza e Pazzi,43.677821,-79.443724,Italian Restaurant


Let's check how many venues were returned for each neighborhood

In [343]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alderwood, Toronto, ON, Canada",8,8,8,8,8,8
"Annex, Toronto, ON, Canada",24,24,24,24,24,24
"Bathurst Manor, Toronto, ON, Canada",5,5,5,5,5,5
"Bay St & Wellesley St W, Toronto, ON, Canada",39,39,39,39,39,39
"Bayview Village, Toronto, ON, Canada",3,3,3,3,3,3
"Bayview Woods-Steeles, Toronto, ON, Canada",6,6,6,6,6,6
"Beaconsfield, Toronto, ON, Canada",41,41,41,41,41,41
"Bedford Park, Grimsby, ON, Canada",2,2,2,2,2,2
"Bendale, Toronto, ON, Canada",5,5,5,5,5,5


Let's find out how many unique categories can be curated from all the returned venues

In [344]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 260 uniques categories.


### Analyze Each Neighborhood

In [345]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Laundromat,Lawyer,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size

In [346]:
toronto_onehot.shape

(3092, 260)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶

In [347]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Laundromat,Lawyer,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Alderwood, Toronto, ON, Canada",0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.25000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000

Let's confirm the new size

In [349]:
toronto_grouped.shape

(108, 260)

Let's print each neighborhood along with the top 5 most common venues

In [350]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood, Toronto, ON, Canada----
                     venue  freq
0              Pizza Place  0.25
1       Athletics & Sports  0.25
2                 Pharmacy  0.25
3              Coffee Shop  0.25
4  North Indian Restaurant  0.00


----Annex, Toronto, ON, Canada----
            venue  freq
0  Sandwich Place  0.12
1            Park  0.08
2             Pub  0.08
3     Pizza Place  0.08
4     Coffee Shop  0.08


----Bathurst Manor, Toronto, ON, Canada----
               venue  freq
0     Baseball Field   0.2
1         Playground   0.2
2               Park   0.2
3        Men's Store   0.2
4  Convenience Store   0.2


----Bay St & Wellesley St W, Toronto, ON, Canada----
                 venue  freq
0          Coffee Shop  0.10
1         Burger Joint  0.08
2  Japanese Restaurant  0.05
3                Diner  0.05
4      Bubble Tea Shop  0.05


----Bayview Village, Toronto, ON, Canada----
                        venue  freq
0                       Trail  0.33
1  Construction & Landscap

                                      venue  freq
0                                      Park   0.5
1  Residential Building (Apartment / Condo)   0.5
2                               Zoo Exhibit   0.0
3                               Music Venue   0.0
4                   North Indian Restaurant   0.0


----Humber Heights, Toronto, ON, Canada----
         venue  freq
0         Park  0.25
1       Bakery  0.25
2   Restaurant  0.25
3         Café  0.25
4  Zoo Exhibit  0.00


----Humber Summit, Toronto, ON, Canada----
                     venue  freq
0             Home Service   1.0
1              Zoo Exhibit   0.0
2              Music Store   0.0
3  North Indian Restaurant   0.0
4             Noodle House   0.0


----Humber Valley Village Junior Middle School, Toronto, ON, Canada----
           venue  freq
0       Pharmacy  0.33
1   Skating Rink  0.17
2           Bank  0.17
3           Park  0.17
4  Grocery Store  0.17


----Humbermede, Toronto, ON, Canada----
              venue  freq
0    

         venue  freq
0  Pizza Place  0.29
1  Coffee Shop  0.14
2  Supermarket  0.14
3     Bus Line  0.14
4         Park  0.14


----Roncesvalles, Toronto, ON, Canada----
               venue  freq
0        Coffee Shop  0.16
1               Park  0.10
2   Sushi Restaurant  0.10
3             Bakery  0.10
4  Food & Drink Shop  0.10


----Rouge, Toronto, ON, Canada----
                  venue  freq
0           Zoo Exhibit  0.56
1          Dessert Shop  0.11
2            Restaurant  0.11
3  Fast Food Restaurant  0.11
4                   Zoo  0.11


----Scarborough Junction, Toronto, ON, Canada----
            venue  freq
0      Playground  0.33
1     Coffee Shop  0.33
2  Discount Store  0.33
3     Zoo Exhibit  0.00
4     Music Venue  0.00


----Scarborough Village, Toronto, ON, Canada----
                 venue  freq
0   Seafood Restaurant  0.12
1              Butcher  0.12
2  Fried Chicken Joint  0.12
3          Pizza Place  0.12
4         Skating Rink  0.12


----Scarborough, Toronto, ON

Let's put that into a pandas dataframe. First, let's write a function to sort the venues in descending order.

In [351]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [352]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alderwood, Toronto, ON, Canada",Athletics & Sports,Pizza Place,Coffee Shop,Pharmacy,Zoo,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
1,"Annex, Toronto, ON, Canada",Sandwich Place,Pizza Place,Park,Pub,Coffee Shop,Metro Station,Grocery Store,Shoe Repair,Café,Liquor Store
2,"Bathurst Manor, Toronto, ON, Canada",Baseball Field,Park,Convenience Store,Playground,Men's Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
3,"Bay St & Wellesley St W, Toronto, ON, Canada",Coffee Shop,Burger Joint,Bubble Tea Shop,Diner,Japanese Restaurant,Italian Restaurant,Hobby Shop,Sandwich Place,Mexican Restaurant,Fast Food Restaurant
4,"Bayview Village, Toronto, ON, Canada",Construction & Landscaping,Trail,Dog Run,Zoo,Fast Food Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Field


##  Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [383]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 1, 0, 0, 1, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [384]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Carleton Village Shore, NS, Canada",43.661584,-65.328253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Corso Italia Garage, Toronto, ON, Canada",43.676946,-79.444991,0.0,Italian Restaurant,Coffee Shop,Café,Breakfast Spot,Mediterranean Restaurant,Bakery,Convenience Store,Pub,Sandwich Place,Mexican Restaurant
2,"York, Toronto, ON, Canada",43.680189,-79.492232,0.0,Pizza Place,Coffee Shop,Supermarket,Beer Store,Zoo,Field,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
3,"Toronto, ON, Canada",43.702303,-79.357423,0.0,Coffee Shop,Café,Hotel,Restaurant,Burger Joint,Bar,Seafood Restaurant,Bakery,Deli / Bodega,Steakhouse
4,"Brookhaven-Amesbury, Toronto, ON, Canada",43.701141,-79.485589,4.0,Park,Zoo,Ethiopian Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field


**NOTE:** There are NaN values upon merging, to aid this issue, all NaN rows will be remove

In [386]:
toronto_merged = toronto_merged.dropna()

In [387]:
toronto_merged.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Corso Italia Garage, Toronto, ON, Canada",43.676946,-79.444991,0.0,Italian Restaurant,Coffee Shop,Café,Breakfast Spot,Mediterranean Restaurant,Bakery,Convenience Store,Pub,Sandwich Place,Mexican Restaurant
2,"York, Toronto, ON, Canada",43.680189,-79.492232,0.0,Pizza Place,Coffee Shop,Supermarket,Beer Store,Zoo,Field,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
3,"Toronto, ON, Canada",43.702303,-79.357423,0.0,Coffee Shop,Café,Hotel,Restaurant,Burger Joint,Bar,Seafood Restaurant,Bakery,Deli / Bodega,Steakhouse
4,"Brookhaven-Amesbury, Toronto, ON, Canada",43.701141,-79.485589,4.0,Park,Zoo,Ethiopian Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field
6,"Weston, Toronto, ON, Canada",43.702465,-79.515471,0.0,Train Station,Coffee Shop,Pizza Place,Skating Rink,Laundromat,Café,Fried Chicken Joint,Discount Store,Diner,Grocery Store


In [388]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

In [389]:
toronto_merged.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Corso Italia Garage, Toronto, ON, Canada",43.676946,-79.444991,0,Italian Restaurant,Coffee Shop,Café,Breakfast Spot,Mediterranean Restaurant,Bakery,Convenience Store,Pub,Sandwich Place,Mexican Restaurant
2,"York, Toronto, ON, Canada",43.680189,-79.492232,0,Pizza Place,Coffee Shop,Supermarket,Beer Store,Zoo,Field,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
3,"Toronto, ON, Canada",43.702303,-79.357423,0,Coffee Shop,Café,Hotel,Restaurant,Burger Joint,Bar,Seafood Restaurant,Bakery,Deli / Bodega,Steakhouse
4,"Brookhaven-Amesbury, Toronto, ON, Canada",43.701141,-79.485589,4,Park,Zoo,Ethiopian Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field
6,"Weston, Toronto, ON, Canada",43.702465,-79.515471,0,Train Station,Coffee Shop,Pizza Place,Skating Rink,Laundromat,Café,Fried Chicken Joint,Discount Store,Diner,Grocery Store


Finally, let's visualize the resulting clusters

In [390]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
         [lat, lon],
         radius=5,
         popup=label,
         color=rainbow[cluster-1],
         fill=True,
         fill_color=rainbow[cluster-1],
         fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


## Examine Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster. 

#### CLUSTER 1

In [392]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,43.676946,Coffee Shop,Café,Breakfast Spot,Mediterranean Restaurant,Bakery,Convenience Store,Pub,Sandwich Place,Mexican Restaurant
2,43.680189,Coffee Shop,Supermarket,Beer Store,Zoo,Field,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
3,43.702303,Café,Hotel,Restaurant,Burger Joint,Bar,Seafood Restaurant,Bakery,Deli / Bodega,Steakhouse
6,43.702465,Coffee Shop,Pizza Place,Skating Rink,Laundromat,Café,Fried Chicken Joint,Discount Store,Diner,Grocery Store
7,43.693114,Park,Pizza Place,Market,Thai Restaurant,Fast Food Restaurant,Bank,Bakery,Japanese Restaurant,Discount Store
8,43.715395,Convenience Store,Sandwich Place,Chinese Restaurant,Fast Food Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Field
9,43.689015,American Restaurant,Fast Food Restaurant,Asian Restaurant,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Field,Fish & Chips Shop
10,43.734119,Pizza Place,Health & Beauty Service,Zoo,Field,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
11,43.758716,Zoo,Ethiopian Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field
12,43.743482,Clothing Store,Hockey Field,Hockey Arena,Latin American Restaurant,Grocery Store,Discount Store,Falafel Restaurant,Farm,Farmers Market


#### CLUSTER 2

In [393]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,43.654064,Flower Shop,Bus Line,Bakery,Business Service,Metro Station,Pool,Lawyer,Fish Market,Flea Market
28,43.686351,Intersection,Zoo,Field,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
29,43.691120,Café,Park,Restaurant,Zoo,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
58,43.775790,Park,Skating Rink,Fast Food Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Zoo,Ethiopian Restaurant
79,43.802603,Park,Fast Food Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Zoo,Electronics Store
80,43.796801,Park,Dog Run,Field,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Zoo,Event Space
85,43.671724,Café,Pool,Event Service,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant
91,43.699591,Pool,Bus Stop,Bridge,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
103,43.796801,Park,Dog Run,Field,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Zoo,Event Space
117,43.760575,Campground,Greek Restaurant,History Museum,Zoo,Farmers Market,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant


#### CLUSTER 3

In [395]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,43.728046,Art Gallery,Filipino Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
87,52.383190,Cosmetics Shop,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field
167,43.728046,Art Gallery,Filipino Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop


#### CLUSTER 4

In [396]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
108,43.680695,Zoo,Food Court,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field


#### CLUSTER 5

In [397]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,43.701141,Zoo,Ethiopian Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field
101,43.736186,Zoo,Ethiopian Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field
139,43.701141,Zoo,Ethiopian Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field
